In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import make_blobs
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import imageio

# Hyperparameters
timesteps = 200
batch_size = 128
epochs = 300
learning_rate = 1e-3

# Generate two Gaussians dataset in corners (4,4) and (4,-4)
X, _ = make_blobs(n_samples=10000, centers=[(4,4), (4,-4)], cluster_std=0.5, random_state=42)
X = torch.tensor(X, dtype=torch.float32)

# Define beta schedule
def linear_beta_schedule(timesteps):
    beta_start = 0.0001
    beta_end = 0.02
    return torch.linspace(beta_start, beta_end, timesteps)

betas = linear_beta_schedule(timesteps)
alphas = 1. - betas
alphas_cumprod = torch.cumprod(alphas, axis=0)
sqrt_alphas_cumprod = torch.sqrt(alphas_cumprod)
sqrt_one_minus_alphas_cumprod = torch.sqrt(1. - alphas_cumprod)

# Helper function to extract the appropriate t index for a batch of indices
def extract(a, t, x_shape):
    batch_size = t.shape[0]
    out = a.gather(-1, t)
    return out.reshape(batch_size, *((1,) * (len(x_shape) - 1)))

# Forward diffusion process
def q_sample(x_start, t, noise=None):
    if noise is None:
        noise = torch.randn_like(x_start)

    sqrt_alphas_cumprod_t = extract(sqrt_alphas_cumprod, t, x_start.shape)
    sqrt_one_minus_alphas_cumprod_t = extract(sqrt_one_minus_alphas_cumprod, t, x_start.shape)

    return sqrt_alphas_cumprod_t * x_start + sqrt_one_minus_alphas_cumprod_t * noise

# Simple MLP model
class SimpleMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(2 + 1, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 2)
        )

    def forward(self, x, t):
        x = torch.cat([x, t.unsqueeze(-1)], dim=-1)
        return self.net(x)

# Training function
def train(model, dataloader, optimizer, device):
    model.train()
    total_loss = 0
    for x in tqdm(dataloader):
        x = x[0].to(device)
        optimizer.zero_grad()

        t = torch.randint(0, timesteps, (x.shape[0],), device=device).long()
        noise = torch.randn_like(x)
        x_noisy = q_sample(x, t, noise)
        noise_pred = model(x_noisy, t.float())
        loss = nn.MSELoss()(noise_pred, noise)

        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(dataloader)

# Sampling function
@torch.no_grad()
def sample(model, n_samples, device, save_path=None):
    model.eval()
    x = torch.randn(n_samples, 2).to(device)
    images = []
    for t in reversed(range(timesteps)):
        t_batch = torch.full((n_samples,), t, device=device, dtype=torch.long)
        predicted_noise = model(x, t_batch.float())
        alpha = alphas[t]
        alpha_hat = alphas_cumprod[t]
        beta = betas[t]
        if t > 0:
            noise = torch.randn_like(x)
        else:
            noise = torch.zeros_like(x)
        x = 1 / torch.sqrt(alpha) * (x - ((1 - alpha) / (torch.sqrt(1 - alpha_hat))) * predicted_noise) + torch.sqrt(beta) * noise
        
        if t % 10 == 0 or t == timesteps - 1:  # Save every 10th frame and the first frame
            fig, ax = plt.subplots(figsize=(6, 6))
            ax.scatter(x.cpu().numpy()[:, 0], x.cpu().numpy()[:, 1], s=1, c='red', alpha=0.7)
            ax.set_xlim(-6, 6)
            ax.set_ylim(-6, 6)
            ax.set_title(f"Step {timesteps - t}/{timesteps}")
            plt.close(fig)
            images.append(fig2img(fig))
    
    if save_path:
        imageio.mimsave(save_path, images, fps=5)
    
    return x

# Helper function to convert matplotlib figure to image
def fig2img(fig):
    fig.canvas.draw()
    return np.array(fig.canvas.renderer._renderer)

# Set up device, model, optimizer, and dataloader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleMLP().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
dataloader = DataLoader(TensorDataset(X), batch_size=batch_size, shuffle=True)

# Training loop
losses = []
for epoch in range(epochs):
    loss = train(model, dataloader, optimizer, device)
    losses.append(loss)
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1}/{epochs} completed, Loss: {loss:.4f}")

# Print final loss
print(f"Final loss: {losses[-1]:.4f}")

# Generate samples and create gif
samples = sample(model, 1000, device, save_path='diffusion_process.gif')

# Plot loss
plt.figure(figsize=(10, 6))
plt.plot(range(1, epochs + 1), losses)
plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.tight_layout()
plt.savefig('training_loss.png')
plt.close()

print("Diffusion process gif saved as 'diffusion_process.gif'")
print("Training loss plot saved as 'training_loss.png'")

  0%|          | 0/79 [00:00<?, ?it/s]

100%|██████████| 79/79 [00:00<00:00, 397.71it/s]


Epoch 10/300 completed, Loss: 0.7787


100%|██████████| 79/79 [00:00<00:00, 315.96it/s]


Epoch 20/300 completed, Loss: 0.6114


...

Epoch 280/300 completed, Loss: 0.4400


100%|██████████| 79/79 [00:00<00:00, 449.06it/s]


Epoch 290/300 completed, Loss: 0.4277


100%|██████████| 79/79 [00:00<00:00, 425.77it/s]


Epoch 300/300 completed, Loss: 0.4365
Final loss: 0.4365
Diffusion process gif saved as 'diffusion_process.gif'
Training loss plot saved as 'training_loss.png'


In [27]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import torch

# Sampling function
@torch.no_grad()
def sample(model, n_samples, device, source_samples, target_samples):
    model.eval()
    x = torch.randn(n_samples, 2).to(device)
    frames = []
    
    fig, ax = plt.subplots(figsize=(10, 10))
    scatter = ax.scatter([], [], s=5, c='red', alpha=0.7, label='Generated')
    source_scatter = ax.scatter(source_samples[:, 0], source_samples[:, 1], s=10, c='blue', alpha=0.3, label='Source')
    target_scatter = ax.scatter(target_samples[:, 0], target_samples[:, 1], s=10, c='green', alpha=0.3, label='Target')
    time_text = ax.text(0.02, 0.02, '', transform=ax.transAxes, fontsize=16, verticalalignment='bottom')
    
    # Set axis limits based on the data
    all_data = np.vstack((source_samples, target_samples))
    x_min, x_max = all_data[:, 0].min(), all_data[:, 0].max()
    y_min, y_max = all_data[:, 1].min(), all_data[:, 1].max()
    margin = 0.1 * max(x_max - x_min, y_max - y_min)
    ax.set_xlim(x_min - margin, x_max + margin)
    ax.set_ylim(y_min - margin, y_max + margin)
    
    ax.legend(loc='lower left')
    
    # Remove axis and ticks
    ax.axis('off')
    
    def update(frame):
        nonlocal x
        t = timesteps - 1 - frame
        t_batch = torch.full((n_samples,), t, device=device, dtype=torch.long)
        predicted_noise = model(x, t_batch.float())
        alpha = alphas[t]
        alpha_hat = alphas_cumprod[t]
        beta = betas[t]
        if t > 0:
            noise = torch.randn_like(x)
        else:
            noise = torch.zeros_like(x)
        x = 1 / torch.sqrt(alpha) * (x - ((1 - alpha) / (torch.sqrt(1 - alpha_hat))) * predicted_noise) + torch.sqrt(beta) * noise
        
        scatter.set_offsets(x.cpu().numpy())
        time_text.set_text(f't = {frame}')
        return scatter, time_text
    
    anim = animation.FuncAnimation(fig, update, frames=timesteps, interval=50, blit=True)
    
    # Save as GIF
    anim.save('diffusion_process.gif', writer='pillow', fps=20)
    plt.close(fig)
    
    return x

# Generate source and target samples
source_samples = torch.randn(1000, 2).numpy()
target_samples = X.numpy()

# Generate samples and create animation
samples = sample(model, 5000, device, source_samples, target_samples)

# Plot loss
plt.figure(figsize=(10, 6))
plt.plot(range(1, epochs + 1), losses)
plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.tight_layout()
plt.savefig('training_loss.png')
plt.close()

print("Diffusion process animation saved as 'diffusion_process.gif'")
print("Training loss plot saved as 'training_loss.png'")

Diffusion process animation saved as 'diffusion_process.gif'
Training loss plot saved as 'training_loss.png'


In [16]:
# Generate samples
samples = sample(model, 5000, device).cpu().numpy()

# Define font sizes
base_fontsize = 18
title_fontsize = base_fontsize + 2
label_fontsize = base_fontsize
tick_fontsize = base_fontsize - 2

# Plot results
plt.figure(figsize=(20, 6))
plt.rcParams.update({'font.size': base_fontsize})  # Set base font size

plt.subplot(131)
plt.scatter(X[:, 0], X[:, 1], s=5, c='blue', alpha=0.7)
plt.title("Original Data", fontsize=title_fontsize)
plt.xticks(fontsize=tick_fontsize)
plt.yticks(fontsize=tick_fontsize)

plt.subplot(132)
plt.scatter(samples[:, 0], samples[:, 1], s=5, c='red', alpha=0.7)
plt.title("Generated Samples", fontsize=title_fontsize)
plt.xticks(fontsize=tick_fontsize)
plt.yticks(fontsize=tick_fontsize)

plt.subplot(133)
plt.plot(range(1, epochs + 1), losses, linewidth=4)
plt.title("Training Loss", fontsize=title_fontsize)
plt.xlabel("Epoch", fontsize=label_fontsize)
plt.ylabel("Loss", fontsize=label_fontsize)
plt.xticks(fontsize=tick_fontsize)
plt.yticks(fontsize=tick_fontsize)

# Add dotted line at minimum loss
min_loss = min(losses)
plt.axhline(y=min_loss, color='r', linestyle=':', linewidth=2)

# Add text for minimum objective using LaTeX
plt.text(epochs/2, min_loss, r'$\mathbf{min~~ L_{\rm diffusion} > 0}$', 
         horizontalalignment='right', verticalalignment='bottom', 
         fontsize=label_fontsize, color='r', fontweight='bold')

plt.tight_layout()
plt.savefig('diffusion_results.png', dpi=300, bbox_inches='tight')
plt.close()